<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/dev/ytvideo_whisper_KMN_BOT_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper 機器狼特別版

這個專案採用了[Whisper](https://github.com/openai/whisper)而不是以前在用的Decipher專案

如果想要用原本的Whisper專案，就在這裡

# 使用方法

1. 請確認你的執行階段有啟用GPU，在 ``執行階段 -> 變更執行階段類型 ``
1. 更改設定
1. 按下 ``執行階段 -> 執行全部`` 或是 (CTRL+F9)
1. 然後連接到Google雲端硬碟
1. 然後等一下你的檔案應該就會在 ``雲端硬碟 -> Whisper -> result``

## 獲得Google雲端硬碟權限
(在 雲端硬碟 -> Whisper)

In [1]:
#@markdown # **下載選項**
#@markdown ---
#@markdown ## **停用Google雲端硬碟**
disable_google_drive = "yes" #@param ['yes', 'no']
# @markdown ## **下載方式** ``anonfile / transfer.sh / 直線下載``
download_method = "transfer.sh" #@param ['anonfile', 'transfer.sh', 'direct']
#@markdown 當啟用 ``結束自動刪除執行階段`` 的時候**無法**使用直線下載，會回到上傳到anonfiles的方式。請見諒 </p>
#@markdown **警告：直線下載有時候無法在Firefox或類Firefox瀏覽器 (WaterFox , Librewolf)使用**。如果無法使用的話，請**切換到上傳檔案到anonfile**。
if disable_google_drive.lower() == 'no':

  from google.colab import drive
  import os

  path = '/content/gdrive/MyDrive/Whisper/'
  path2 = '/content/gdrive/MyDrive/Whisper/result'
  drive.mount('/content/gdrive')

  if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path2)


# 安裝程式
openai-whisper yt-dlp ipython transfersh_client pyperclip wget

In [2]:
! apt-get install ffmpeg
! pip install openai-whisper yt-dlp ipython transfersh_client pyperclip wget

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.9 MB/s eta 0:00:0

# 測試GPU是否有開啟

如果沒有東西跑出來的話，請去 ``執行階段 -> 變更執行階段類型`` 更改

In [3]:
! nvidia-smi

Thu Aug  3 07:28:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 設定


In [4]:
import os
#@markdown # **設定**
#@markdown ---
#@markdown ## **影片連接**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=FbyV0TwL-wg" #@param {type:"string"}
# File Name Change
#@markdown ## **檔案名稱**
new_filename = "How to make X to be Twitter" #@param {type:"string"}
#@markdown ## **語言**
language = "English" #@param ["Auto", "Chinese", "English", "Japanese"]
#@markdown ## **模式 ``生成 / 翻譯``**
task = "transcribe" #@param ['transcribe', 'translate']
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + " Transcript.txt"
# Change the Model of the Whisper
#@markdown # **其他選項**
#@markdown ---
#@markdown ## **生產逐字稿**
Generate_Plain_Document = "yes" #@param ['yes','no']
#@markdown ## **結束自動刪除執行階段**
shutdown_after_complete = "yes" #@param ['yes','no']
#@markdown ## **模型**
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']
#@markdown ## **Cookies驗證**
enable_cookies = "no" #@param ['yes', 'no']

# 如果你有啟動Cookies驗證
請在這裡上傳Cookies檔案

In [5]:
import os
if enable_cookies.lower() == "yes":
  print("It is not recommended to upload your main account's cookies files here, as this is a shared environment.")
  print("Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.")
  print("The recommended approach is to use another Google Account's cookies to run this tool.")
  print("If it is a private video, you can share the video with that Google Account to grant access to it.")
  print("")

  from google.colab import files
  cookies = files.upload()

  # Specify the path where the files are located
  path_cookies = "/content"

  def rename_files(path_cookies):
      for filename in os.listdir(path_cookies):
          file_path = os.path.join(path_cookies, filename)
          if os.path.isfile(file_path) and filename.endswith(".txt"):
              new_path = os.path.join(path_cookies, "cookies.txt")
              os.rename(file_path, new_path)

  rename_files(path_cookies)

# 下載影片
使用yt-dlp跟ffmpeg

In [6]:
import yt_dlp

if enable_cookies.lower() == "yes":

  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
          'cookiefile': 'cookies.txt'
  }

else:

    ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
  }

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=FbyV0TwL-wg
[youtube] FbyV0TwL-wg: Downloading webpage
[youtube] FbyV0TwL-wg: Downloading ios player API JSON
[youtube] FbyV0TwL-wg: Downloading android player API JSON
[youtube] FbyV0TwL-wg: Downloading player 2363d0d2
[youtube] FbyV0TwL-wg: Downloading m3u8 information
[info] FbyV0TwL-wg: Downloading 1 format(s): 251
[download] Destination: content/audio
[download] 100% of    2.76MiB in 00:00:00 at 17.12MiB/s  
[ExtractAudio] Destination: content/audio.mp3
Deleting original file content/audio (pass -k to keep)


# 生成字幕檔

In [7]:
import whisper

model = whisper.load_model(model_user)

print("Whisper model loaded.")


from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"
if language.lower() == 'Auto':
  result = model.transcribe(input_file, verbose=True, task=task)
else:
  result = model.transcribe(input_file, verbose=True, task=task, language=language)

if disable_google_drive.lower() == 'no':

  transcription_root = "/content/gdrive/MyDrive/Whisper/result"

else:
  os.mkdir("result")
  transcription_root = "/content/result"

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, new_filename)

if Generate_Plain_Document.lower() == "yes":

  # Save as TXT file
  srt_writer = get_writer("txt", transcription_root,)
  srt_writer(result, new_filename2)
  print("Generated srt and txt file in the Google Drive -> Whisper -> result Folder")

else:

  print("Generated srt file in the Google Drive -> Whisper -> result Folder")

100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 88.0MiB/s]


Whisper model loaded.
[00:00.000 --> 00:05.280]  Today, we are looking at X.
[00:05.280 --> 00:12.960]  And this is the social media, originally called Twitter, but now it's called X.
[00:12.960 --> 00:17.080]  Which have a nice domain called X.com.
[00:17.080 --> 00:20.640]  The most easy domain that I can remember is in my brain.
[00:20.640 --> 00:25.680]  So as we can see, this is our X.
[00:25.680 --> 00:30.680]  And it don't have a nice blue bird that we all remember.
[00:30.680 --> 00:33.360]  But what can I do?
[00:33.360 --> 00:36.280]  Is enable this extension right here.
[00:36.280 --> 00:38.480]  And I could refresh.
[00:38.480 --> 00:39.960]  Everything is back to Twitter.
[00:39.960 --> 00:42.040]  Our blue bird is back to life.
[00:42.040 --> 00:46.200]  Now, a W says, how can you install this extension?
[00:46.200 --> 00:51.880]  First thing, if you are a Firefox user, well, you can already go to Firefox marketplace.
[00:51.880 --> 00:53.840]  You can already get this ex

# 壓縮檔案並上傳
如果雲端硬碟被停用了

In [8]:
import shutil
import os.path
import requests
from google.colab import files
from transfersh_client.app import send_to_transfersh, create_zip, remove_file

if disable_google_drive.lower() == 'yes':
    # Creating the ZIP file
    archived = shutil.make_archive(new_filename, 'zip', '/content/result')

    if os.path.exists(archived):
        print(archived)

        if shutdown_after_complete.lower() == "yes":
            if download_method.lower() == "anonfiles":
              # Upload the file to Anonfiles
              upload_url = "https://api.anonfiles.com/upload"
              with open(archived, 'rb') as file:
                  response = requests.post(upload_url, files={'file': file})

              # Check if the upload was successful
              if response.status_code == 200 and response.json()['status']:
                  download_url = response.json()['data']['file']['url']['full']
                  print(f"File uploaded successfully to Anonfiles. Download link: {download_url}")
              else:
                  print("File upload to Anonfiles failed.")
            else:
              # Upload to transfer.sh
              def send_files_from_dir():
                directory = '/content'
                send_to_transfersh(archived, clipboard=False)  # sends archive to transfer.sh


              if __name__ == '__main__':
                send_files_from_dir()


        elif download_method.lower() == "anonfile":
            # Upload the file to Anonfiles
            upload_url = "https://api.anonfiles.com/upload"
            with open(archived, 'rb') as file:
                response = requests.post(upload_url, files={'file': file})

            # Check if the upload was successful
            if response.status_code == 200 and response.json()['status']:
                download_url = response.json()['data']['file']['url']['full']
                print(f"File uploaded successfully to Anonfiles. Download link: {download_url}")
            else:
                print("File upload to Anonfiles failed.")

        elif download_method.lower() == "transfer.sh":
            # Upload to transfer.sh
            def send_files_from_dir():
              directory = '/content'
              send_to_transfersh(archived, clipboard=False)  # sends archive to transfer.sh


            if __name__ == '__main__':
              send_files_from_dir()

        else:
            files.download(archived)
            print("File downloaded successfully.")

    else:
        print("ZIP file not created")

/content/How to make X to be Twitter.srt.zip

Sending file: How to make X to be Twitter.srt.zip (size of the file: 0.003002 MB)
Link to download file(will be saved till 2023-08-17):
https://transfer.sh/qy0Izsb0Ai/How%20to%20make%20X%20to%20be%20Twitter.srt.zip



# 自動關閉執行階段

In [ ]:
if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()